In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [2]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['RealBackground1_07092017.txt', 'RealBackground2_07092017.txt', 'RealBackground3_07092017.txt', 'RealBackground4_07092017.txt', 'RealBackground5_07092017.txt', 'RealBackground6_07092017.txt', 'RealBackground7_07092017.txt', 'RealBackground8_07092017.txt', 'RealBackground9_07092017.txt', 'RealBackground10_07092017.txt', 'RealBackground_Al_1_07092017.txt', 'RealBackground_Al_2_07092017.txt', 'RealBackground_Al_3_07092017.txt', 'RealBackground_Al_4_07092017.txt', 'RealBackground_Al_5_07092017.txt', 'RealBackground_Al_6_07092017.txt', 'RealBackground_Al_7_07092017.txt', 'RealBackground_Al_8_07092017.txt', 'RealBackground_Al_9_07092017.txt', 'RealBackground_Al_10_07092017.txt']


In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [5]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (417*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay2 = []
    for i in range(0,len(x)):
        timedelay = -((x[i,0]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay2.append(timedelay)
    return(timeDelay2)

In [ ]:
PDvalues = pd.Series()
times2 = pd.Series()
PDvalues2 = pd.Series()
AvgPD10_2 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,900):    
    time = frame[1][i]
    times2 = times2.append(time)
    PDvalue = frame[3][i]
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues)) 
PDvalues = PDvalues.values.reshape(900, len(allFiles))
PDvaluesTHzBlocked = PDvalues[:,1:10].astype(np.float)
PDvaluesTHzBlockedAl = PDvalues[:,11:20].astype(np.float)
times2 = times2.values.reshape(900,len(allFiles)) 

timeDelay2 = ConvertToTimeDelay(times2,itr,offset) # function converts to timedelay
#print(timeDelay2)


# PDvalues2 = PDvalues2.astype(np.float) #Ensures all the values are floats so that I can plot them!
AvgTHzBlocked = PDvaluesTHzBlocked.mean(axis = 1)
AvgPD_THzBlockedAl = PDvaluesTHzBlockedAl.mean(axis = 1)
#print(len(timeDelay2))
label1 = 'THz pump Laser blocked'
label2 = 'THz blocked with Al'
plt.plot(timeDelay2,AvgTHzBlocked,'r.',label = label1)
plt.plot(timeDelay2,AvgPD_THzBlockedAl,'b.',label = label2)
plt.xlabel('Stage Delay (ps)')
plt.ylabel('PD Signal (arb)')
plt.legend(loc = 'center')
plt.grid(True)
plt.show()